# Whatstics

### This script summarizes a WhatsApp chat history.

<br>

🚨🚨🚨🚨🚨 CAUTION 🚨🚨🚨🚨🚨

The script use openai api and may cost you A LOT OF MONEY. Please be aware of that.

<br>

# 1. First configure Date limits and the paths

For example:
- START_DATE = "2024-08-23"
- END_DATE = "2024-08-23"
- PATH = "../history/chat.txt"
- OUTPUT_PATH = "../summary/output.txt"

Start and end date are inclusive, and must be equal

🚨🚨🚨🚨🚨 CAUTION 🚨🚨🚨🚨🚨

Bigger the date range, more expensive the cost.

In [1]:
START_DATE = "2024-08-19"
END_DATE = "2024-08-20"

FILE_PATH = "../history/conversa.txt"
SUMMARY_PATH = "../summary"

In [2]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


True

In [3]:
import pandas as pd
import re


def add_mesage_to_df(messages, date, time, id, message, type_="message", subtype=None):
    messages.append(
        {
            "Date": date,
            "Time": time,
            "Id": id,
            "Message": message,
            "Type": type_,
            "Action_type": subtype,
        }
    )
    return messages


def manage_action(messages, current_date, current_time, id, rest_of_line):
    if "entrou usando o link de convite deste grupo" in rest_of_line:
        action_type = "entry"
        id = rest_of_line.split("entrou usando o link de convite deste grupo")[
            0
        ].strip()
    elif "saiu" in rest_of_line:
        action_type = "exit"
        id = rest_of_line.split("saiu")[0].strip()
    elif "mudou a descrição do grupo" in rest_of_line:
        action_type = "description_change"
        id = rest_of_line.split("mudou a descrição do grupo")[0].strip()
    elif "fixou uma mensagem" in rest_of_line:
        action_type = "message_pin"
        id = rest_of_line.split("fixou uma mensagem")[0].strip()
    elif "mudou as configurações" in rest_of_line:
        action_type = "group_settings_change"
        id = rest_of_line.split("mudou as configurações do grupo")[0].strip()
    elif "(arquivo anexado)" in rest_of_line:
        action_type = "file_attach"
        id = rest_of_line.split("(arquivo anexado)")[0].strip()
    elif "criou o grupo" in rest_of_line:
        action_type = "group_create"
        id = rest_of_line.split("criou o grupo")[0].strip()
    elif "foi adicionado(a)" in rest_of_line:
        action_type = "add"
        id = rest_of_line.split("foi adicionado(a)")[0].strip()
    else:
        print("Error line:", rest_of_line)
        assert False

    add_mesage_to_df(
        messages, current_date, current_time, id, None, "action", subtype=action_type
    )


def parse_whatsapp_history(file_path):
    # Lista para armazenar as mensagens extraídas
    messages = []

    # Ler o arquivo de histórico
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    # Regex para identificar a data, hora e número do remetente
    date_and_time_pattern = re.compile(r"(\d{2}/\d{2}/\d{4}) (\d{2}:\d{2}) - (.*)")

    # Variáveis para acumular mensagens
    current_date = None
    current_time = None
    current_id = None
    current_message = None

    for line in lines:
        # Verifica se a linha corresponde ao padrão de uma nova mensagem
        match = date_and_time_pattern.match(line)
        if match:
            if current_message:
                messages = add_mesage_to_df(
                    messages, current_date, current_time, current_id, current_message
                )
            current_message = ""
            current_date, current_time, rest_of_line = match.groups()
            if "‎" in rest_of_line:
                manage_action(
                    messages, current_date, current_time, current_id, rest_of_line
                )
            else:
                if ":" in rest_of_line:
                    current_id, message = rest_of_line.split(":", 1)
                    current_message = message.strip()
        else:
            current_message += "\n" + line.strip()

    messages = add_mesage_to_df(
        messages, current_date, current_time, current_id, current_message, "message"
    )

    df = pd.DataFrame(messages)

    df["Order"] = range(1, len(df) + 1)
    
    df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str))

    return df

/home/bruno/anaconda3/envs/hug/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
# Exemplo de uso
history = parse_whatsapp_history(FILE_PATH)
history[history["Type"] != "message"].head()

/tmp/ipykernel_85047/1342128291.py:101: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str))


,Date,Time,Id,Message,Type,Action_type,Order,DateTime
0,19/08/2024,19:53,‎~ Felipe Soares,None,action,group_create,1,2024-08-19 19:53:00
4,19/08/2024,20:32,‎+55 61 9217-6339,None,action,entry,5,2024-08-19 20:32:00
6,19/08/2024,20:33,‎+55 11 98910-2316,None,action,entry,7,2024-08-19 20:33:00
30,19/08/2024,20:39,‎+55 11 98760-9956,None,action,entry,31,2024-08-19 20:39:00
65,19/08/2024,20:44,‎+55 13 97406-4831,None,action,entry,66,2024-08-19 20:44:00


In [5]:
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2626 entries, 0 to 2625
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         2626 non-null   object        
 1   Time         2626 non-null   object        
 2   Id           2626 non-null   object        
 3   Message      2540 non-null   object        
 4   Type         2626 non-null   object        
 5   Action_type  86 non-null     object        
 6   Order        2626 non-null   int64         
 7   DateTime     2626 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 164.3+ KB


In [6]:
print("Min DateTime:", history["DateTime"].min())
print("Max DateTime:", history["DateTime"].max())

Min DateTime: 2024-08-19 19:53:00
Max DateTime: 2024-08-24 01:26:00


In [7]:
from datetime import datetime, timedelta

# Convert strings to datetime objects
start_datetime = datetime.strptime(START_DATE, "%Y-%m-%d")
end_datetime = datetime.strptime(END_DATE, "%Y-%m-%d") + timedelta(days=1)

print(start_datetime, end_datetime)

messages_df = history[
    (history["Type"] == "message")
    & (history["DateTime"] >= start_datetime)
    & (history["DateTime"] < end_datetime)
]
print(len(messages_df))
messages_df.tail()

2024-08-19 00:00:00 2024-08-21 00:00:00
787


,Date,Time,Id,Message,Type,Action_type,Order,DateTime
849,20/08/2024,23:28,+55 11 93056-8112,<Mídia oculta>,message,None,850,2024-08-20 23:28:00
850,20/08/2024,23:31,+55 19 98377-7511,"É exatamente disso que a gente precisa, recome...",message,None,851,2024-08-20 23:31:00
851,20/08/2024,23:32,+55 19 98377-7511,Ninguém vive só de Python e R,message,None,852,2024-08-20 23:32:00
852,20/08/2024,23:32,+55 31 9232-2516,Esse é qualidade https://www.instagram.com/ros...,message,None,853,2024-08-20 23:32:00
853,20/08/2024,23:41,+55 19 98377-7511,Anotado,message,None,854,2024-08-20 23:41:00


In [8]:
print("Min DateTime:", messages_df["DateTime"].min())
print("Max DateTime:", messages_df["DateTime"].max())

Min DateTime: 2024-08-19 20:32:00
Max DateTime: 2024-08-20 23:41:00


In [9]:
from pprint import pprint

# Full chat string
# Every new message starts with a new line followed by NM:
full_chat_str = " ".join(messages_df["Message"].values)

full_chat_str[1000:2000]

'n: https://www.linkedin.com/in/edimilson-ribeiro/ Boa noite :) Vai ser incrível Pessoal, acabei de promover 3 pessoas para me ajudar administrar o grupo.\nEm breve vou criar um servidor para nos no Discord.\n\n@5511938006662  @553484415505  @553175701040 São jose SC Recomendo o podcast do Hipster e do Papo de IA\nhttps://open.spotify.com/show/3lyrnNKGPqdVgmsBUPxvmf?si=1c7c4708e02a4348 Eu larguei  minha carreia de 6 anos como modelo internacional pra focar na tecnologia  vamos ver se vai dá certo isso Boa noite! Sou Medico Veterinário apaixonado por inovações em dados, vai ser massa. Palotina-PR <Mensagem editada> Boa noite pessoal. Sou Maurício, moro em Novo Hamburgo RS. Sou Economista quero me especializar em AI. Bora 2 anos é logo ali! Discord muito bom viu\n\nDependendo da pra reunir pra estudar junto e debater\n\nAté pela alura tbm Show Durmo tarde.e.acordo.cedo\n.\n.tô.em.casa.e cheia de energia.\n\n\nNo que eu puder.ajudar estou às ordens... @5511972856203 depois podemos falar c

In [10]:

from pprint import pprint

# prompts from: https://github.com/omer-go/group_chat_summarizer
# Many thanks!

start_datetime_prompt =  start_datetime.strftime("%d/%m/%Y")
end_datetime_prompt = datetime.strptime(END_DATE, "%Y-%m-%d").strftime("%d/%m/%Y")

print(start_datetime_prompt, end_datetime_prompt)

date_range_prompt = ""
if start_datetime_prompt != end_datetime_prompt:
    date_range_prompt = f"de {start_datetime_prompt} a {end_datetime_prompt}"
else:
    date_range_prompt = f"em {start_datetime_prompt}"

# SUMMARY_PROMPT = f"""
# Por favor, resuma o seguinte bate-papo de grupo do WhatsApp com base nos tópicos discutidos. As conversas ocorreram {date_range_prompt}. Para cada tópico, inclua seu título e um resumo em tópicos. Os tópicos devem incluir informações detalhadas e relevantes para nossa turma de Graduação em Inteligência Artificial. Inclua no resumo todas as recomendações sobre empresas, serviços específicos, vagas de emprego, recomendações de ferramentas computacionais, recursos educacionais e contatos (links de redes sociais, email ou telefones). Inclua também todos os links que foram compartilhados.
# """


# RANKING_PROMPT = """
# Analise as mensagens do grupo de WhataApp do curso de Graduação em Inteligência Artifial.
# Classifique os assuntos discutidos com base na relevância para a turma.
# Considere a importância das informações compartilhadas, a frequência com que foram discutidas e a quantidade de interações que geraram.
# Informações consideradas importantes podem incluir: contatos (LinkedIn, redes sociais, emails e telefones), recursos educacionais (links, playlists do youtube, livros, cursos etc), softwares, 
# Por favor, retorne apenas uma lista com um título para as categorias mais relevantes, sem nenhuma informação adicional
# """

RANKING_PROMPT = """
Analise as mensagens do grupo de WhataApp do curso de Graduação em Inteligência Artificial. 
Classifique os assuntos discutidos com base na relevância para a turma.
Considere a importância das informações compartilhadas, a frequência com que foram discutidas e a quantidade de interações que geraram.
Informações consideradas importantes podem incluir: contatos (LinkedIn, redes sociais, emails e telefones), recursos educacionais (links, playlists do youtube e outros, livros, cursos etc), softwares, notas de aula, vagas de emprego, recomendações de empresas, serviços específicos, informações acadêmicas (eventos, datas limites, prazos, trabalhos, atividades de classe) ou quaisquer outros tópicos que sejam relevantes. 
Por favor, retorne apenas uma lista com um título para as categorias mais relevantes (máximo de 8 categorias), sem nenhuma informação adicional.
"""


SUMMARY_PROMPT = """
Você receberá um histórico de conversas do WhatsApp. Para cada um dos seguintes tópicos: "{topics}", analise as mensagens e
faça um resumo em subtópicos do que discutido. Mostre todos os links que foram compartilhados sobre cada tópico, não deixe de exibir nenhum link.
"""


# SUMMARY_PROMPT = f"""
# Por favor, resuma o seguinte bate-papo em grupo do WhatsApp, ocorrido {date_range_prompt}, destacando os tópicos discutidos de forma detalhada e relevante para nossa turma de Graduação em Inteligência Artificial. Para cada tópico, forneça um título claro e organize o conteúdo em pontos resumidos, incluindo:

# 1. **Recomendações**: Detalhe qualquer recomendação de empresas, serviços específicos, vagas de emprego, ferramentas computacionais, recursos educacionais e contatos compartilhados (incluindo links de redes sociais, e-mails ou telefones).
# 2. **Links Compartilhados**: Inclua todos os links que foram compartilhados, especificando seu contexto e relevância.
# 3. **Discussões Gerais**: Capture informações importantes e conclusões discutidas, especialmente aquelas relacionadas ao nosso curso e área de estudo.

# Certifique-se de que o resumo seja claro, abrangente e útil para os membros do grupo.
# """



NEWSLETTER_PROMPT = f"""Por favor, forneça um parágrafo para abrir uma newsletter cobrindo os seguintes tópicos:"""

pprint(RANKING_PROMPT)

19/08/2024 20/08/2024
('\n'
 'Analise as mensagens do grupo de WhataApp do curso de Graduação em '
 'Inteligência Artificial. \n'
 'Classifique os assuntos discutidos com base na relevância para a turma.\n'
 'Considere a importância das informações compartilhadas, a frequência com que '
 'foram discutidas e a quantidade de interações que geraram.\n'
 'Informações consideradas importantes podem incluir: contatos (LinkedIn, '
 'redes sociais, emails e telefones), recursos educacionais (links, playlists '
 'do youtube e outros, livros, cursos etc), softwares, notas de aula, vagas de '
 'emprego, recomendações de empresas, serviços específicos, informações '
 'acadêmicas (eventos, datas limites, prazos, trabalhos, atividades de classe) '
 'ou quaisquer outros tópicos que sejam relevantes. \n'
 'Por favor, retorne apenas uma lista com um título para as categorias mais '
 'relevantes (máximo de 8 categorias), sem nenhuma informação adicional.\n')


In [11]:
def print_model(model):
    print("Uging model: ")
    print(model.dict())
    print('\n\n')

In [12]:
import langchain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# from langchain_core.output_parsers import JsonOutputParser

# llm = ChatOpenAI(model="gpt-4o")
llm = ChatOpenAI(model="gpt-4o-mini")
print_model(llm)

ranking_prompt = PromptTemplate(
    template="{instruction}\nAs mensagens: ```\{messages}```",
    input_variables=["instruction", "messages"],
)

ranking_chain = ranking_prompt | llm | StrOutputParser()


langchain.debug = False

ranking_response = ranking_chain.invoke(
    {
        "instruction": RANKING_PROMPT,
        "messages": full_chat_str,
    }
)

pprint(ranking_response)

Uging model: 
{'model_name': 'gpt-4o-mini', 'model': 'gpt-4o-mini', 'stream': False, 'n': 1, 'temperature': 0.7, '_type': 'openai-chat'}



('1. Networking e Contatos Profissionais\n'
 '2. Recursos Educacionais e Materiais de Estudo\n'
 '3. Informações Acadêmicas e Eventos do Curso\n'
 '4. Vagas de Emprego e Oportunidades Profissionais\n'
 '5. Discussões sobre Linguagens de Programação e Ferramentas\n'
 '6. Criação e Organização de Grupos de Estudo\n'
 '7. Suporte e Colaboração entre Alunos\n'
 '8. Experiências e Histórias Pessoais dos Alunos')


In [13]:
# summary_prompt = PromptTemplate(
#     template="{instruction}\nAs mensagens: ```\{messages}```",
#     input_variables=["instruction", "messages", "topins"],
# )

# summary_chain = (
#     {"topics": ranking_chain, "instruction": SUMMARY_PROMPT, "messages": full_chat_str}
#     | summary_prompt
#     | llm
# )

# langchain.debug = True

# summary_chain

In [14]:

SUMMARY_PROMPT = """
Você receberá um histórico de conversas do WhatsApp. Para cada um dos seguintes tópicos: "{topics}", analise as mensagens e
faça um resumo em subtópicos do que discutido. Mostre todos os links que foram compartilhados sobre cada tópico.
"""

summary_instruction = SUMMARY_PROMPT.format(topics=ranking_response)
pprint(summary_instruction)


('\n'
 'Você receberá um histórico de conversas do WhatsApp. Para cada um dos '
 'seguintes tópicos: "1. Networking e Contatos Profissionais\n'
 '2. Recursos Educacionais e Materiais de Estudo\n'
 '3. Informações Acadêmicas e Eventos do Curso\n'
 '4. Vagas de Emprego e Oportunidades Profissionais\n'
 '5. Discussões sobre Linguagens de Programação e Ferramentas\n'
 '6. Criação e Organização de Grupos de Estudo\n'
 '7. Suporte e Colaboração entre Alunos\n'
 '8. Experiências e Histórias Pessoais dos Alunos", analise as mensagens e\n'
 'faça um resumo em subtópicos do que discutido. Mostre todos os links que '
 'foram compartilhados sobre cada tópico.\n')


In [15]:

summary_prompt = PromptTemplate(
    template="{instruction}\nAs mensagens: ```\{messages}```",
    input_variables=["instruction", "messages"],
)

summary_chain = ranking_prompt | llm | StrOutputParser()

langchain.debug = False

summary_response = summary_chain.invoke({
    "instruction": summary_instruction,
    "messages": full_chat_str,
})

pprint(summary_response)

('Aqui está o resumo das discussões do histórico de mensagens do WhatsApp, '
 'organizado por tópicos:\n'
 '\n'
 '### 1. Networking e Contatos Profissionais\n'
 '- Diversos participantes compartilharam seus perfis do LinkedIn para se '
 'conectar e fortalecer a rede profissional.\n'
 '- Links do LinkedIn compartilhados:\n'
 '  - [Edimilson Ribeiro](https://www.linkedin.com/in/edimilson-ribeiro/)\n'
 '  - [Roberto Besser](https://www.linkedin.com/in/robertobesser/)\n'
 '  - [Gabriel](https://www.linkedin.com/in/gvfa/)\n'
 '  - [Johnatan '
 'Loriano](https://www.linkedin.com/in/johnatanloriano?utm_source=share&utm_campaign=share_via&utm_content=profile&utm_medium=ios_app)\n'
 '  - [Felipe '
 'Soares](https://www.linkedin.com/in/digitalmanagerfelipesoares/)\n'
 '  - [Reinaldo '
 'Moura](https://www.linkedin.com/in/reinaldo-moura-7aba27295?utm_source=share&utm_campaign=share_via&utm_content=profile&utm_medium=android_app)\n'
 '  - [Henrique Sarmento](https://www.linkedin.com/in/hen-sarment

In [16]:
# copy to clipboard:
import pyperclip
pyperclip.copy(summary_response)

In [17]:
start_range = datetime.strptime(START_DATE, "%Y-%m-%d").strftime("%d-%m-%Y")
end_range = datetime.strptime(END_DATE, "%Y-%m-%d").strftime("%d-%m-%Y")

if start_range == end_range:
    date_range_filename = f"/summary_whatsapp_{start_range}.txt"
else:
    date_range_filename = f"/summary_whatsapp_from_{start_range}_to_{end_range}.txt"

# save to file
with open(SUMMARY_PATH + date_range_filename, "w") as file:
    file.write(summary_response)